In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras import layers, Model

# Load dataset
data = pd.read_excel('Tweets.xlsx')

# Separate columns
tweets = data['Tweet']
targets = data['Target']
stances = data['Stance']
train_test = data['Train/Test']

# Encode labels
target_encoder = LabelEncoder()
stance_encoder = LabelEncoder()

encoded_targets = target_encoder.fit_transform(targets)
encoded_stances = stance_encoder.fit_transform(stances)

# Train/Test split
train_indices = train_test == 'Train'
test_indices = train_test == 'Test'

X_train = tweets[train_indices]
X_test = tweets[test_indices]
y_train_target = encoded_targets[train_indices]
y_train_stance = encoded_stances[train_indices]
y_test_target = encoded_targets[test_indices]
y_test_stance = encoded_stances[test_indices]

# Tokenization with BERT tokenizer
bert_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

def tokenize_texts(texts, max_len=50):
    return tokenizer(
        list(texts),
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )

X_train_tokenized = tokenize_texts(X_train)
X_test_tokenized = tokenize_texts(X_test)

/Users/macbook/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
from tensorflow.keras import layers, Model

# Load BERT model and tokenizer
bert_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = TFBertModel.from_pretrained(bert_model_name)

# Custom Layer for BERT
class BertLayer(layers.Layer):
    def __init__(self, bert_model):
        super(BertLayer, self).__init__()
        self.bert = bert_model

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state[:, 0, :]  # CLS token

# Input layers
input_ids = layers.Input(shape=(50,), dtype=tf.int32, name="input_ids")
attention_mask = layers.Input(shape=(50,), dtype=tf.int32, name="attention_mask")

# Pass through custom BERT layer
cls_token = BertLayer(bert_model)([input_ids, attention_mask])

# Fully connected layers
x = layers.Dense(128, activation='relu')(cls_token)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.3)(x)

# Output layer
output = layers.Dense(6, activation='softmax', name="stance_output")(x)  # Adjust number of classes as needed

# Define the model
model = Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary
print(model.summary())

# Train the model
history = model.fit(
    {
        'input_ids': X_train_tokenized['input_ids'],
        'attention_mask': X_train_tokenized['attention_mask']
    },
    y_train_target,
    validation_split=0.2,
    epochs=100,
    batch_size=16,
    verbose=1
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_ids           │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_mask      │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_layer          │ (None, 768)       │          0 │ input_ids[0][0],  │
│ (BertLayer)         │                   │            │ attention_mask[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     98,432 │ bert_layer[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stance_output       │ (None, 6)         │        390 │ dropout_1[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 107,078 (418.27 KB)

 Trainable params: 107,078 (418.27 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 229s 2s/step - accuracy: 0.1994 - loss: 1.8766 - val_accuracy: 0.3156 - val_loss: 1.6715
Epoch 2/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 227s 2s/step - accuracy: 0.2690 - loss: 1.7300 - val_accuracy: 0.3328 - val_loss: 1.6292
Epoch 3/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 222s 2s/step - accuracy: 0.2728 - loss: 1.6921 - val_accuracy: 0.3688 - val_loss: 1.5987
Epoch 4/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 223s 2s/step - accuracy: 0.3088 - loss: 1.6381 - val_accuracy: 0.4305 - val_loss: 1.5659
Epoch 5/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 223s 2s/step - accuracy: 0.2943 - loss: 1.6243 - val_accuracy: 0.4545 - val_loss: 1.5421
Epoch 6/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 230s 2s/step - accuracy: 0.2984 - loss: 1.6084 - val_accuracy: 0.4734 - val_loss: 1.5089
Epoch 7/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 229s 2s/step - accuracy: 0.3678 - loss: 1.5496 - val_accuracy: 0.5077 - val_loss: 1.4797
Epoch 8/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 228s 2s/step - accuracy: 0.3624 - loss: 1.54

In [4]:
print(type(input_ids), type(attention_mask))

<class 'keras.src.backend.common.keras_tensor.KerasTensor'> <class 'keras.src.backend.common.keras_tensor.KerasTensor'>


In [ ]:
model_stance = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(np.unique(y_train_stance)), activation='softmax')  # Stance sınıfları kadar çıktı
])

model_stance.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model_stance.summary())

# Eğitim
model_stance.fit(
    X_train_padded,
    y_train_stance,
    validation_split=0.2,
    batch_size=32,
    epochs=10,
    verbose=1
)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_10                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 6s 54ms/step - accuracy: 0.4541 - loss: 1.0757 - val_accuracy: 0.4768 - val_loss: 1.0251
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - accuracy: 0.5982 - loss: 0.8603 - val_accuracy: 0.5026 - val_loss: 1.0036
Epoch 3/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - accuracy: 0.8242 - loss: 0.4315 - val_accuracy: 0.5815 - val_loss: 1.1309
Epoch 4/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - accuracy: 0.9605 - loss: 0.1424 - val_accuracy: 0.5935 - val_loss: 1.6154
Epoch 5/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - accuracy: 0.9879 - loss: 0.0483 - val_accuracy: 0.5695 - val_loss: 1.9164
Epoch 6/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 69ms/step - accuracy: 0.9992 - loss: 0.0138 - val_accuracy: 0.5849 - val_loss: 2.0321
Epoch 7/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - accuracy: 0.9975 - loss: 0.0109 - val_accuracy: 0.5815 - val_loss: 2.2519
Epoch 8/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - accuracy: 0.9988 - loss: 0.0062 - val_accuracy: 0.605

In [5]:
# Evaluate the model
loss, accuracy = model.evaluate(
    {
        'input_ids': X_test_tokenized['input_ids'],
        'attention_mask': X_test_tokenized['attention_mask']
    },
    y_test_target
)

print(f"Test Accuracy: {accuracy}")



21/62 ━━━━━━━━━━━━━━━━━━━━ 2:00 3s/step - accuracy: 0.4116 - loss: 3.0924

KeyboardInterrupt: 

In [7]:
print(f"Test Accuracy")

Test Accuracy


In [ ]:
# Stance Model Değerlendirme
stance_loss, stance_accuracy = model_stance.evaluate(X_test_padded, y_test_stance, verbose=1)
print(f"Stance Model - Test Loss: {stance_loss}, Test Accuracy: {stance_accuracy}")


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
